# Proje
## Birliktelik Kuralı Temelli Tavsiye Sistemi

## İş Problemi
* Sepet aşamasındaki kullanıcalara ürün önerisinde bulunmak

## Veri Ön İşleme

In [4]:
#!pip install mlxtend

In [5]:
import pandas as pd

In [6]:
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)
#çıktının tek bir satırda olmasını sağlar 
pd.set_option("display.expand_frame_repr", None) 

In [7]:
from mlxtend.frequent_patterns import apriori, association_rules

In [8]:
df_ = pd.read_excel("/Users/huseyinefkanalp/Desktop/DataScience/DataSets/online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [9]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


In [11]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [12]:
df.shape

(541910, 8)

In [13]:
def outlier_thresholds(df, var):
    q1 = df[var].quantile(0.01)
    q3 = df[var].quantile(0.99)
    interq_range = q3 - q1
    up_limit = q3 + 1.5 * interq_range
    low_limit = q1 - 1.5 * interq_range
    return low_limit, up_limit

In [14]:
def replace_with_thresholds(df, var):
    low_limit, up_limit = outlier_thresholds(df,var)
    df.loc[(df[var]<low_limit), var] = low_limit
    df.loc[(df[var]>up_limit), var] = up_limit

In [15]:
def retail_data_prep(df):
    df.dropna(inplace = True)
    df = df[~df["Invoice"].str.contains("C", na = False)]
    df = df[df["Quantity"]>0]
    df = df[df["Price"]>0]
    replace_with_thresholds(df,"Quantity")
    replace_with_thresholds(df,"Price")
    return df

In [16]:
df = retail_data_prep(df)
df.shape

(397885, 8)

In [17]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0,11.830770,25.523052,1.000,2.00,6.00,12.00,298.50
Price,397885.0,2.893492,3.227175,0.001,1.25,1.95,3.75,37.06
Customer ID,397885.0,15294.416882,1713.144421,12346.000,13969.00,15159.00,16795.00,18287.00


In [18]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

## ARL Veri Yapısını Hazırlama (Invoice-Product Matrix)

In [19]:
df_fr = df[df["Country"] == "France"]
df_fr.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.0,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.0,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.0,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.0,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24.0,2010-12-01 08:45:00,0.65,12583.0,France


In [23]:
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).head()

Quantity
Invoice Description                               
536370   SET 2 TEA TOWELS I LOVE LONDON       24.0
        ALARM CLOCK BAKELIKE GREEN            12.0
        ALARM CLOCK BAKELIKE PINK             24.0
        ALARM CLOCK BAKELIKE RED              24.0
        CHARLOTTE BAG DOLLY GIRL DESIGN       20.0

In [26]:
# unstack kullanarak
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 NaN                NaN                          NaN                      NaN                             24.0
536852                                 NaN                NaN                          NaN                      NaN                              NaN
536974                                 NaN                NaN                          NaN                      NaN                              NaN
537065                                 NaN                NaN                          NaN                      NaN                              NaN
537463                                 NaN                NaN                          NaN                      NaN                              NaN

In [28]:
#Nan değerleri 0 a çevirme
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 0.0                0.0                          0.0                      0.0                             24.0
536852                                 0.0                0.0                          0.0                      0.0                              0.0
536974                                 0.0                0.0                          0.0                      0.0                              0.0
537065                                 0.0                0.0                          0.0                      0.0                              0.0
537463                                 0.0                0.0                          0.0                      0.0                              0.0

In [30]:
#applymap kullanarak 1 ve 0 değerler ile doldurma
df_fr.groupby(["Invoice","Description"]).agg({"Quantity": "sum"})\
.unstack().fillna(0).applymap(lambda x : 1 if x > 0 else 0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                   0                  0                            0                        0                                1
536852                                   0                  0                            0                        0                                0
536974                                   0                  0                            0                        0                                0
537065                                   0                  0                            0                        0                                0
537463                                   0                  0                            0                        0                                0

In [56]:
def create_invoice_product_df(df, idd=False):
    if idd:
        return df.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"})\
        .unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0)
    else:
        return df.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"})\
        .unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0)
    

In [57]:
fr_inv = create_invoice_product_df(df_fr, idd=True)
fr_inv .head()

Quantity                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Birliktelik Kurallarının Çıkarılması

apriori fonksiyonu ile olası tüm ürün birlikteliklerinin
support değerlerini (olasılıklarını) bulmak
* apriori der ki:
 Bana df ver, çalışmanın başında belirlemek istediğin 
 eşik değerin varsa ver ve kullanmış olduğun verisetindeki değişken isimlerini kullanmak istiyorsan use_colnames = True yap

In [44]:
# from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(fr_inv, min_support = 0.01, use_colnames = True)

/Users/huseyinefkanalp/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [45]:
frequent_itemsets.sort_values("support", ascending = False)

,support,itemsets
538,0.773779,"((Quantity, POST))"
387,0.187661,"((Quantity, 23084))"
107,0.179949,"((Quantity, 21731))"
243,0.172237,"((Quantity, 22554))"
245,0.169666,"((Quantity, 22556))"
...,...,...
18793,0.010283,"((Quantity, 22551), (Quantity, 21086), (Quanti..."
18787,0.010283,"((Quantity, 22326), (Quantity, 23256), (Quanti..."
18786,0.010283,"((Quantity, 22326), (Quantity, 22728), (Quanti..."
18785,0.010283,"((Quantity, 22326), (Quantity, 22727), (Quanti..."


In [46]:
# Birliktelik kurallarını çıkartma

In [49]:
rules = association_rules(frequent_itemsets, metric = "support", min_threshold = 0.01)

In [50]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"((Quantity, 21791))","((Quantity, 10002))",0.028278,0.020566,0.010283,0.363636,17.681818,0.009701,1.539111
1,"((Quantity, 10002))","((Quantity, 21791))",0.020566,0.028278,0.010283,0.500000,17.681818,0.009701,1.943445
2,"((Quantity, 21915))","((Quantity, 10002))",0.069409,0.020566,0.010283,0.148148,7.203704,0.008855,1.149771
3,"((Quantity, 10002))","((Quantity, 21915))",0.020566,0.069409,0.010283,0.500000,7.203704,0.008855,1.861183
4,"((Quantity, 22551))","((Quantity, 10002))",0.136247,0.020566,0.010283,0.075472,3.669811,0.007481,1.059388


* antecedents : Önceki Ürün
* consequents : İkinci Ürün
* antecedent support : İlk ürünün tek başına gözlenme olasılığı
* consequent support : İkinci ürünün tek başına gözlenme olasılığı
* support : İki ürünün birlikte gözükme olasılığı
* confidence : X ürünü alındığında Y ürününün alınma olasılığı
* lift : X ürünü satın alındığında Y ürününün satılma olasığı .. kat artmaktadır
* leverage : 
* conviction : X ürünü olmadan Y ürününün beklenen frekansıdır yada tam tersi

In [54]:
rules[(rules["support"]>0.05)&(rules["confidence"]>0.1)&(rules["lift"]>5)]\
.sort_values("confidence", ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23706,"((Quantity, 21094), (Quantity, 21080))","((Quantity, 21086))",0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301
23708,"((Quantity, 21080), (Quantity, 21086))","((Quantity, 21094))",0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612
108823,"((Quantity, POST), (Quantity, 21086), (Quantit...","((Quantity, 21094))",0.084833,0.128535,0.082262,0.969697,7.544242,0.071358,28.758355
108821,"((Quantity, 21094), (Quantity, POST), (Quantit...","((Quantity, 21086))",0.084833,0.138817,0.082262,0.969697,6.985410,0.070486,28.419023
1776,"((Quantity, 21094))","((Quantity, 21086))",0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
...,...,...,...,...,...,...,...,...,...
7213,"((Quantity, 22629))","((Quantity, 22630))",0.125964,0.100257,0.071979,0.571429,5.699634,0.059351,2.099400
62247,"((Quantity, 22630))","((Quantity, POST), (Quantity, 22629))",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443
62246,"((Quantity, POST), (Quantity, 22629))","((Quantity, 22630))",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443
62249,"((Quantity, 22629))","((Quantity, 22630), (Quantity, POST))",0.125964,0.074550,0.053985,0.428571,5.748768,0.044594,1.619537


## Sonuç
Buradan çıkan sonuç ile eğer müşteri antecedent değişkenindeki ürünü satın aldıysa
consequents değişkenindeki ürünü de almasını önereceğiz

## Çalışmanın Scriptini Hazırlama

In [ ]:
def outlier_thresholds(df, var):
    q1 = df[var].quantile(0.01)
    q3 = df[var].quantile(0.99)
    interq_range = q3 - q1
    up_limit = q3 + 1.5 * interq_range
    low_limit = q1 - 1.5 * interq_range
    return low_limit, up_limit

In [ ]:
def replace_with_thresholds(df, var):
    low_limit, up_limit = outlier_thresholds(df,var)
    df.loc[(df[var]<low_limit), var] = low_limit
    df.loc[(df[var]>up_limit), var] = up_limit

In [ ]:
def retail_data_prep(df):
    df.dropna(inplace = True)
    df = df[~df["Invoice"].str.contains("C", na = False)]
    df = df[df["Quantity"]>0]
    df = df[df["Price"]>0]
    replace_with_thresholds(df,"Quantity")
    replace_with_thresholds(df,"Price")
    return df

In [58]:
def create_invoice_product_df(df, idd=False):
    if idd:
        return df.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"})\
        .unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0)
    else:
        return df.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"})\
        .unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0)
    

In [59]:
def check_id(df, stock_code):
    product_name = df[df["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

In [61]:
def create_rules(df, idd = True, country ="France"):
    df = df[df["Country"] == country]
    df = create_invoice_product_df(df,idd)
    frequent_itemset = apriori(df, min_support=0.01, use_colnames = True)
    rules = association_rules(frequent_itemset, metric="support", min_threshold=0.01)
    return rules

In [62]:
df = df_.copy()

In [63]:
df = retail_data_prep(df)

In [64]:
rules = create_rules(df)

/Users/huseyinefkanalp/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [66]:
rules[(rules["support"]>0.05)&(rules["confidence"]>0.1)&(rules["lift"]>5)]\
.sort_values("confidence", ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23706,"((Quantity, 21094), (Quantity, 21080))","((Quantity, 21086))",0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301
23708,"((Quantity, 21080), (Quantity, 21086))","((Quantity, 21094))",0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612
108823,"((Quantity, POST), (Quantity, 21086), (Quantit...","((Quantity, 21094))",0.084833,0.128535,0.082262,0.969697,7.544242,0.071358,28.758355
108821,"((Quantity, 21094), (Quantity, POST), (Quantit...","((Quantity, 21086))",0.084833,0.138817,0.082262,0.969697,6.985410,0.070486,28.419023
1776,"((Quantity, 21094))","((Quantity, 21086))",0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
...,...,...,...,...,...,...,...,...,...
7213,"((Quantity, 22629))","((Quantity, 22630))",0.125964,0.100257,0.071979,0.571429,5.699634,0.059351,2.099400
62247,"((Quantity, 22630))","((Quantity, POST), (Quantity, 22629))",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443
62246,"((Quantity, POST), (Quantity, 22629))","((Quantity, 22630))",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443
62249,"((Quantity, 22629))","((Quantity, 22630), (Quantity, POST))",0.125964,0.074550,0.053985,0.428571,5.748768,0.044594,1.619537


## Sepet Aşamasındaki Kullanıcılara Ürün Önermesinde Bulunmak

In [68]:
# Örnek : 
# Kullanıcı örnek ürün id: 22492
product_id = 22492

In [70]:
check_id(df, product_id)

['MINI PAINT SET VINTAGE ']


In [71]:
sorted_rules = rules.sort_values("lift", ascending = False)

In [86]:
recommendation_list = []
for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        for k in list(j):
            if k == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

In [96]:
recommendation_list[0:10]

[('Quantity', 22382),
 ('Quantity', 22382),
 ('Quantity', 22382),
 ('Quantity', 22963),
 ('Quantity', 22962),
 ('Quantity', 22726),
 ('Quantity', 22556),
 ('Quantity', 21094),
 ('Quantity', 22554),
 ('Quantity', 22551)]

In [97]:
check_id(df, 22382)

['LUNCH BAG SPACEBOY DESIGN ']
